In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as sqlio
import requests as re
from IPython.display import JSON
import json

In [2]:
flights = pd.read_csv('../../../data/df_train.csv')

In [3]:
airport_link = pd.read_csv('../../../data/airport_link.csv')

In [4]:
airport_link.drop(['Unnamed: 0'],axis=1, inplace=True)

In [5]:
airport_link.columns

Index(['origin_airport_id', 'origin_city_name', 'dest_airport_id',
       'dest_city_name'],
      dtype='object')

In [6]:
origin_airport = airport_link[['origin_airport_id','origin_city_name']]

In [7]:
dest_airport = airport_link[['dest_airport_id', 'dest_city_name']]

In [8]:
origins = origin_airport.drop_duplicates()
origins

,origin_airport_id,origin_city_name
0,13930,"Chicago, IL"
1,14747,"Seattle, WA"
2,12892,"Los Angeles, CA"
5,11057,"Charlotte, NC"
6,11292,"Denver, CO"
14,14107,"Phoenix, AZ"
15,14771,"San Francisco, CA"
16,10397,"Atlanta, GA"
26,11298,"Dallas/Fort Worth, TX"
27,12266,"Houston, TX"


In [9]:
dest = dest_airport.drop_duplicates()
dest

,dest_airport_id,dest_city_name
0,15356,"Trenton, NJ"
1,10423,"Austin, TX"
3,12892,"Los Angeles, CA"
4,13487,"Minneapolis, MN"
5,13930,"Chicago, IL"
...,...,...
174068,14905,"Santa Maria, CA"
174622,13087,"Las Vegas, NV"
176686,10918,"Cedar City, UT"
177802,12898,"Latrobe, PA"


In [10]:
flights = flights.drop('Unnamed: 0',axis=1)

In [11]:
flights

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay
0,12892,10140,10,11,2018,1235,120.0,677,3.454152,4.566613,...,0,0,0,0,0,0,0,0,1,6.0
1,12892,10140,10,11,2018,1840,105.0,677,3.454152,4.566613,...,0,0,0,0,0,0,0,0,1,1.0
2,12892,10397,10,11,2018,805,250.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,0,0,1,-12.0
3,12892,10397,10,11,2018,1045,260.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,0,0,1,-20.0
4,12892,10423,10,11,2018,1030,185.0,1242,3.454152,4.566613,...,0,0,0,0,0,0,0,0,1,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,31,1,2018,1508,87.0,447,4.221989,2.348741,...,0,0,0,0,0,0,0,1,0,-9.0
3918191,14771,14679,31,1,2018,1720,87.0,447,4.221989,2.348741,...,0,0,0,0,0,0,0,1,0,-11.0
3918192,14771,14679,31,1,2018,2055,83.0,447,4.221989,2.348741,...,0,0,0,0,0,0,0,1,0,-14.0
3918193,12892,14771,31,1,2018,2000,94.0,337,4.221989,2.348741,...,0,0,0,0,0,0,0,1,0,-18.0


In [12]:
df = pd.merge(flights, 
                 origins, 
                 on ='origin_airport_id', 
                 how ='left')

In [13]:
df

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,origin_city_name
0,12892,10140,10,11,2018,1235,120.0,677,3.454152,4.566613,...,0,0,0,0,0,0,0,1,6.0,"Los Angeles, CA"
1,12892,10140,10,11,2018,1840,105.0,677,3.454152,4.566613,...,0,0,0,0,0,0,0,1,1.0,"Los Angeles, CA"
2,12892,10397,10,11,2018,805,250.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,0,1,-12.0,"Los Angeles, CA"
3,12892,10397,10,11,2018,1045,260.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,0,1,-20.0,"Los Angeles, CA"
4,12892,10423,10,11,2018,1030,185.0,1242,3.454152,4.566613,...,0,0,0,0,0,0,0,1,-12.0,"Los Angeles, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,31,1,2018,1508,87.0,447,4.221989,2.348741,...,0,0,0,0,0,0,1,0,-9.0,"San Francisco, CA"
3918191,14771,14679,31,1,2018,1720,87.0,447,4.221989,2.348741,...,0,0,0,0,0,0,1,0,-11.0,"San Francisco, CA"
3918192,14771,14679,31,1,2018,2055,83.0,447,4.221989,2.348741,...,0,0,0,0,0,0,1,0,-14.0,"San Francisco, CA"
3918193,12892,14771,31,1,2018,2000,94.0,337,4.221989,2.348741,...,0,0,0,0,0,0,1,0,-18.0,"Los Angeles, CA"


In [14]:
df = pd.merge(df, 
                 dest, 
                 on ='dest_airport_id', 
                 how ='left')

In [15]:
df

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,origin_city_name,dest_city_name
0,12892,10140,10,11,2018,1235,120.0,677,3.454152,4.566613,...,0,0,0,0,0,0,1,6.0,"Los Angeles, CA","Albuquerque, NM"
1,12892,10140,10,11,2018,1840,105.0,677,3.454152,4.566613,...,0,0,0,0,0,0,1,1.0,"Los Angeles, CA","Albuquerque, NM"
2,12892,10397,10,11,2018,805,250.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Atlanta, GA"
3,12892,10397,10,11,2018,1045,260.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,1,-20.0,"Los Angeles, CA","Atlanta, GA"
4,12892,10423,10,11,2018,1030,185.0,1242,3.454152,4.566613,...,0,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Austin, TX"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,31,1,2018,1508,87.0,447,4.221989,2.348741,...,0,0,0,0,0,1,0,-9.0,"San Francisco, CA","San Diego, CA"
3918191,14771,14679,31,1,2018,1720,87.0,447,4.221989,2.348741,...,0,0,0,0,0,1,0,-11.0,"San Francisco, CA","San Diego, CA"
3918192,14771,14679,31,1,2018,2055,83.0,447,4.221989,2.348741,...,0,0,0,0,0,1,0,-14.0,"San Francisco, CA","San Diego, CA"
3918193,12892,14771,31,1,2018,2000,94.0,337,4.221989,2.348741,...,0,0,0,0,0,1,0,-18.0,"Los Angeles, CA","San Francisco, CA"


In [16]:
flights = df.copy()

In [17]:
flights

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,origin_city_name,dest_city_name
0,12892,10140,10,11,2018,1235,120.0,677,3.454152,4.566613,...,0,0,0,0,0,0,1,6.0,"Los Angeles, CA","Albuquerque, NM"
1,12892,10140,10,11,2018,1840,105.0,677,3.454152,4.566613,...,0,0,0,0,0,0,1,1.0,"Los Angeles, CA","Albuquerque, NM"
2,12892,10397,10,11,2018,805,250.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Atlanta, GA"
3,12892,10397,10,11,2018,1045,260.0,1947,3.454152,4.566613,...,0,0,0,0,0,0,1,-20.0,"Los Angeles, CA","Atlanta, GA"
4,12892,10423,10,11,2018,1030,185.0,1242,3.454152,4.566613,...,0,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Austin, TX"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,31,1,2018,1508,87.0,447,4.221989,2.348741,...,0,0,0,0,0,1,0,-9.0,"San Francisco, CA","San Diego, CA"
3918191,14771,14679,31,1,2018,1720,87.0,447,4.221989,2.348741,...,0,0,0,0,0,1,0,-11.0,"San Francisco, CA","San Diego, CA"
3918192,14771,14679,31,1,2018,2055,83.0,447,4.221989,2.348741,...,0,0,0,0,0,1,0,-14.0,"San Francisco, CA","San Diego, CA"
3918193,12892,14771,31,1,2018,2000,94.0,337,4.221989,2.348741,...,0,0,0,0,0,1,0,-18.0,"Los Angeles, CA","San Francisco, CA"


In [18]:
flights['origin_city_name'].unique()

array(['Los Angeles, CA', 'Phoenix, AZ', 'Denver, CO', 'Seattle, WA',
       'San Francisco, CA', 'Atlanta, GA', 'Charlotte, NC',
       'Dallas/Fort Worth, TX', 'Houston, TX', 'Chicago, IL'],
      dtype=object)

In [19]:
citieNames = []
for index, city in enumerate(flights.origin_city_name):
    if '/' in city:
        splits = city.split(sep='/')
        city = splits[0] + ',' + splits[-1].split(sep=',')[-1]
    city2 = city.replace(" ","")
    city3 = city2.replace(",", "")    
    citieNames.append(city3)

In [20]:
flights['origin_city'] = citieNames

In [22]:
flights

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,origin_city_name,dest_city_name,origin_city
0,12892,10140,10,11,2018,1235,120.0,677,3.454152,4.566613,...,0,0,0,0,0,1,6.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
1,12892,10140,10,11,2018,1840,105.0,677,3.454152,4.566613,...,0,0,0,0,0,1,1.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
2,12892,10397,10,11,2018,805,250.0,1947,3.454152,4.566613,...,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
3,12892,10397,10,11,2018,1045,260.0,1947,3.454152,4.566613,...,0,0,0,0,0,1,-20.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
4,12892,10423,10,11,2018,1030,185.0,1242,3.454152,4.566613,...,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Austin, TX",LosAngelesCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,31,1,2018,1508,87.0,447,4.221989,2.348741,...,0,0,0,0,1,0,-9.0,"San Francisco, CA","San Diego, CA",SanFranciscoCA
3918191,14771,14679,31,1,2018,1720,87.0,447,4.221989,2.348741,...,0,0,0,0,1,0,-11.0,"San Francisco, CA","San Diego, CA",SanFranciscoCA
3918192,14771,14679,31,1,2018,2055,83.0,447,4.221989,2.348741,...,0,0,0,0,1,0,-14.0,"San Francisco, CA","San Diego, CA",SanFranciscoCA
3918193,12892,14771,31,1,2018,2000,94.0,337,4.221989,2.348741,...,0,0,0,0,1,0,-18.0,"Los Angeles, CA","San Francisco, CA",LosAngelesCA


In [23]:
flights.head()

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,origin_city_name,dest_city_name,origin_city
0,12892,10140,10,11,2018,1235,120.0,677,3.454152,4.566613,...,0,0,0,0,0,1,6.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
1,12892,10140,10,11,2018,1840,105.0,677,3.454152,4.566613,...,0,0,0,0,0,1,1.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
2,12892,10397,10,11,2018,805,250.0,1947,3.454152,4.566613,...,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
3,12892,10397,10,11,2018,1045,260.0,1947,3.454152,4.566613,...,0,0,0,0,0,1,-20.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
4,12892,10423,10,11,2018,1030,185.0,1242,3.454152,4.566613,...,0,0,0,0,0,1,-12.0,"Los Angeles, CA","Austin, TX",LosAngelesCA


In [24]:
flights.drop(['origin_city_name'],axis=1,inplace=True)

In [52]:
Fflights = flights.copy()

In [54]:
Fflights = Fflights.sort_values(['origin_city', 'year', 'month', 'day', 'crs_dep_time'])


pandas.core.frame.DataFrame

In [55]:
Fflights

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN,arr_delay,dest_city_name,origin_city
338803,10397,11292,1,1,2018,530,205.0,1199,4.221989,4.566613,...,0,0,0,0,0,0,1,-7.0,"Denver, CO",AtlantaGA
338837,10397,12953,1,1,2018,600,125.0,762,4.221989,4.566613,...,0,0,0,0,0,0,1,1.0,"New York, NY",AtlantaGA
338851,10397,13232,1,1,2018,600,115.0,591,4.221989,4.566613,...,0,0,0,0,0,0,1,49.0,"Chicago, IL",AtlantaGA
1265910,10397,13303,1,1,2018,600,115.0,594,4.221989,7.880984,...,0,0,0,0,0,0,0,-12.0,"Miami, FL",AtlantaGA
1266589,10397,14100,1,1,2018,600,124.0,666,4.221989,7.880984,...,0,0,0,0,0,0,0,-27.0,"Philadelphia, PA",AtlantaGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682925,14747,11298,31,12,2019,2320,220.0,1660,5.605429,2.328271,...,0,0,0,0,0,0,0,-19.0,"Dallas/Fort Worth, TX",SeattleWA
2051290,14747,12264,31,12,2019,2330,280.0,2306,5.605429,7.982953,...,0,0,0,0,1,0,0,-18.0,"Washington, DC",SeattleWA
3682770,14747,10299,31,12,2019,2330,224.0,1448,5.605429,2.328271,...,0,0,0,0,0,0,0,-3.0,"Anchorage, AK",SeattleWA
2050716,14747,12266,31,12,2019,2359,248.0,1874,5.605429,7.982953,...,0,0,0,0,1,0,0,-18.0,"Houston, TX",SeattleWA


In [50]:
Fflights

In [57]:
Windspeeds = np.empty(3918195, dtype=object)
weatherDesc = np.empty(3918195, dtype=object)
precipitations = np.empty(3918195, dtype=object)
visibilities = np.empty(3918195, dtype=object)

temp_city = 'AtlantaGA'
r = open("../../../data/weather/201801AtlantaGA.json", "r")
jsonfile = json.load(r)['data']['weather']

temp_month = 1
for i in range(0,len(Fflights)-1):
    row = Fflights.iloc[i,:]
    
    day = row.day
    month = row.month
    city = row.origin_city
    year = row.year
    time = row.crs_dep_time
    
    if month < 10:
        strmonth = '0' + str(month)
    else:
        strmonth = str(month)
        
    if city != temp_city or month != temp_month:
        r.close()
        r = open("../../../data/weather/"+str(year)+strmonth+city+".json", "r")
        try:
            jsonfile = json.load(r)['data']['weather']
        except:
            Windspeeds[i]= np.nan
            visibilities[i]= np.nan
            precipitations[i]= np.nan
            weatherDesc[i]=np.nan
            continue
        temp_city = city
        temp_month = month
    
    if time >= 2100:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][7]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][7]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][7]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][7]['weatherDesc'][0]['value'])
    elif time >= 1800:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][6]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][6]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][6]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][6]['weatherDesc'][0]['value'])
    elif time >= 1500:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][5]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][5]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][5]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][5]['weatherDesc'][0]['value'])
    elif time >= 1200:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][4]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][4]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][4]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][4]['weatherDesc'][0]['value'])
    elif time >= 900:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][3]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][3]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][3]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][3]['weatherDesc'][0]['value'])
    elif time >= 600:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][2]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][2]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][2]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][2]['weatherDesc'][0]['value'])
    elif time >= 300:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][1]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][1]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][1]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][1]['weatherDesc'][0]['value'])
    else:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][0]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][0]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][0]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][0]['weatherDesc'][0]['value'])

In [58]:
Windspeeds

array(['8', '7', '7', ..., '16', '16', None], dtype=object)

In [59]:
Windspeeds = pd.DataFrame(Windspeeds)

In [68]:
weatherDesc = pd.DataFrame(weatherDesc)

In [62]:
visibilities = pd.DataFrame(visibilities)

In [63]:
precipitations = pd.DataFrame(precipitations)

In [71]:
precipitations.isna().sum()

0    19858
dtype: int64

In [72]:
Fflights['windspeeds'] = Windspeeds

In [75]:
Fflights['weather_desc'] = weatherDesc

In [76]:
Fflights['visibilities'] = visibilities

In [77]:
Fflights['precipitations'] = precipitations

In [163]:
Fflights.columns

Index(['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'weather_desc', 'visibilities',
       'precipitations'],
      dtype='object')

In [115]:
Fflights.to_csv('../../../data/final_flights_unclean.csv')

array([7, 4, 1])

In [179]:
train_flights = read_csv('../../../data/final_flights_unclean.csv')

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,carrier_UA,carrier_VX,carrier_WN,arr_delay,dest_city_name,origin_city,windspeeds,weather_desc,visibilities,precipitations
338803,10397,11292,1,1,2018,530,205.0,1199,4.221989,4.566613,...,0,0,1,-7.0,"Denver, CO",AtlantaGA,3,Clear,10,0.0
338837,10397,12953,1,1,2018,600,125.0,762,4.221989,4.566613,...,0,0,1,1.0,"New York, NY",AtlantaGA,3,Clear,10,0.0
338851,10397,13232,1,1,2018,600,115.0,591,4.221989,4.566613,...,0,0,1,49.0,"Chicago, IL",AtlantaGA,3,Clear,10,0.0
1265910,10397,13303,1,1,2018,600,115.0,594,4.221989,7.880984,...,0,0,0,-12.0,"Miami, FL",AtlantaGA,10,Sunny,10,0.0
1266589,10397,14100,1,1,2018,600,124.0,666,4.221989,7.880984,...,0,0,0,-27.0,"Philadelphia, PA",AtlantaGA,12,Sunny,10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682925,14747,11298,31,12,2019,2320,220.0,1660,5.605429,2.328271,...,0,0,0,-19.0,"Dallas/Fort Worth, TX",SeattleWA,4,Partly cloudy,10,0.0
2051290,14747,12264,31,12,2019,2330,280.0,2306,5.605429,7.982953,...,1,0,0,-18.0,"Washington, DC",SeattleWA,5,Sunny,10,0.0
3682770,14747,10299,31,12,2019,2330,224.0,1448,5.605429,2.328271,...,0,0,0,-3.0,"Anchorage, AK",SeattleWA,4,Sunny,10,0.0
2050716,14747,12266,31,12,2019,2359,248.0,1874,5.605429,7.982953,...,1,0,0,-18.0,"Houston, TX",SeattleWA,3,Sunny,10,0.0


In [189]:
train_flights = pd.get_dummies(Fflights,prefix=['weather'], columns = ['weather_desc'], drop_first=True)

In [209]:
train_flights.columns = train_flights.columns.str.replace(' ', '_')

In [211]:
train_flights

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,weather_Patchy_light_rain,weather_Patchy_light_rain_with_thunder,weather_Patchy_light_snow,weather_Patchy_moderate_snow,weather_Patchy_rain_possible,weather_Patchy_sleet_possible,weather_Patchy_snow_possible,weather_Sunny,weather_Thundery_outbreaks_possible,weather_Torrential_rain_shower
338803,10397,11292,1,1,2018,530,205.0,1199,4.221989,4.566613,...,0,0,0,0,0,0,0,0,0,0
338837,10397,12953,1,1,2018,600,125.0,762,4.221989,4.566613,...,0,0,0,0,0,0,0,0,0,0
338851,10397,13232,1,1,2018,600,115.0,591,4.221989,4.566613,...,0,0,0,0,0,0,0,0,0,0
1265910,10397,13303,1,1,2018,600,115.0,594,4.221989,7.880984,...,0,0,0,0,0,0,0,1,0,0
1266589,10397,14100,1,1,2018,600,124.0,666,4.221989,7.880984,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682925,14747,11298,31,12,2019,2320,220.0,1660,5.605429,2.328271,...,0,0,0,0,0,0,0,0,0,0
2051290,14747,12264,31,12,2019,2330,280.0,2306,5.605429,7.982953,...,0,0,0,0,0,0,0,1,0,0
3682770,14747,10299,31,12,2019,2330,224.0,1448,5.605429,2.328271,...,0,0,0,0,0,0,0,1,0,0
2050716,14747,12266,31,12,2019,2359,248.0,1874,5.605429,7.982953,...,0,0,0,0,0,0,0,1,0,0


In [212]:
train_flights['windspeeds'] = pd.to_numeric(train_flights['windspeeds'], downcast='float')
train_flights['visibilities'] = pd.to_numeric(train_flights['visibilities'], downcast='float')
train_flights['precipitations'] = pd.to_numeric(train_flights['precipitations'], downcast='float')


In [213]:
ffull = train_flights.dropna()

In [222]:
f_null = train_flights[train_flights.isnull().any(axis=1)]

In [215]:
ffull.columns

Index(['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'visibilities', 'precipitations',
       'weather_Blowing_snow', 'weather_Clear', 'weather_Cloudy',
       'weather_Fog', 'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_sn

In [217]:
f_avgs = ffull[['origin_city', 'month','windspeeds', 'visibilities', 'precipitations',
       'weather_Blowing_snow', 'weather_Clear', 'weather_Cloudy',
       'weather_Fog', 'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_snow_showers', 'weather_Mist',
       'weather_Moderate_or_heavy_freezing_rain',
       'weather_Moderate_or_heavy_rain_shower',
       'weather_Moderate_or_heavy_rain_with_thunder',
       'weather_Moderate_or_heavy_sleet',
       'weather_Moderate_or_heavy_snow_showers',
       'weather_Moderate_or_heavy_snow_with_thunder', 'weather_Moderate_rain',
       'weather_Moderate_rain_at_times', 'weather_Moderate_snow',
       'weather_Overcast', 'weather_Partly_cloudy',
       'weather_Patchy_heavy_snow', 'weather_Patchy_light_drizzle',
       'weather_Patchy_light_rain', 'weather_Patchy_light_rain_with_thunder',
       'weather_Patchy_light_snow', 'weather_Patchy_moderate_snow',
       'weather_Patchy_rain_possible', 'weather_Patchy_sleet_possible',
       'weather_Patchy_snow_possible', 'weather_Sunny',
       'weather_Thundery_outbreaks_possible',
       'weather_Torrential_rain_shower']]

In [248]:
cm_avg = f_avgs.groupby(['origin_city', 'month']).mean().reset_index()

In [249]:
cm_avg

,origin_city,month,windspeeds,visibilities,precipitations,weather_Blowing_snow,weather_Clear,weather_Cloudy,weather_Fog,weather_Freezing_fog,...,weather_Patchy_light_rain,weather_Patchy_light_rain_with_thunder,weather_Patchy_light_snow,weather_Patchy_moderate_snow,weather_Patchy_rain_possible,weather_Patchy_sleet_possible,weather_Patchy_snow_possible,weather_Sunny,weather_Thundery_outbreaks_possible,weather_Torrential_rain_shower
0,AtlantaGA,1,5.437081,9.667908,0.250812,0.000213,0.169929,0.038258,0.000071,0.0,...,0.005070,0.003723,0.000727,0.000390,0.031504,0.000000,0.000000,0.478779,0.000940,0.002606
1,AtlantaGA,2,5.776533,9.691983,0.298305,0.000000,0.174480,0.049906,0.000230,0.0,...,0.000958,0.003409,0.000364,0.000555,0.055939,0.000134,0.000230,0.428856,0.002892,0.008082
2,AtlantaGA,3,5.712039,9.871572,0.089203,0.000000,0.242552,0.020395,0.001279,0.0,...,0.000224,0.001375,0.000112,0.000000,0.022249,0.000000,0.000048,0.571495,0.000847,0.000384
3,AtlantaGA,4,5.842941,9.665402,0.282090,0.000066,0.185189,0.029779,0.001187,0.0,...,0.001979,0.001303,0.000000,0.000066,0.039327,0.000099,0.000033,0.462355,0.003265,0.003891
4,AtlantaGA,5,6.819535,9.780849,0.135973,0.000000,0.212060,0.022801,0.002763,0.0,...,0.000442,0.001421,0.000537,0.000347,0.034912,0.000000,0.000395,0.542546,0.004042,0.000395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,SeattleWA,8,4.965475,9.141848,0.426558,0.000000,0.110709,0.079440,0.000427,0.0,...,0.009470,0.000789,0.004143,0.002663,0.050110,0.000066,0.000000,0.301680,0.003913,0.002466
116,SeattleWA,9,4.783157,9.318157,0.254997,0.000000,0.148362,0.058144,0.002508,0.0,...,0.005890,0.001900,0.001330,0.002204,0.063046,0.000684,0.000380,0.416014,0.001558,0.002280
117,SeattleWA,10,4.258893,9.733904,0.187828,0.000000,0.211504,0.037333,0.001351,0.0,...,0.002739,0.000750,0.000750,0.002251,0.023826,0.000000,0.000675,0.498949,0.000375,0.002964
118,SeattleWA,11,6.771852,9.623393,0.438821,0.000000,0.206234,0.029150,0.001375,0.0,...,0.000566,0.004043,0.000000,0.000000,0.047505,0.000000,0.000000,0.486537,0.002749,0.009097


In [241]:
f_null.columns

Index(['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'visibilities', 'precipitations',
       'weather_Blowing_snow', 'weather_Clear', 'weather_Cloudy',
       'weather_Fog', 'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_sn

In [242]:
f_n = f_null[['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city']]

In [250]:
fnf = pd.merge(f_n, 
                 cm_avg, 
                 on =['origin_city', 'month'], 
                 how ='left')

In [251]:
fnf.columns

Index(['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'visibilities', 'precipitations',
       'weather_Blowing_snow', 'weather_Clear', 'weather_Cloudy',
       'weather_Fog', 'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_sn

In [261]:
atll = fnf.loc[fnf['origin_city'] == 'AtlantaGA'] 
atll[['month','windspeeds','visibilities','precipitations']]


,month,windspeeds,visibilities,precipitations
0,7,6.706493,9.776554,0.222143
1,7,6.706493,9.776554,0.222143
2,7,6.706493,9.776554,0.222143
3,7,6.706493,9.776554,0.222143
4,7,6.706493,9.776554,0.222143
...,...,...,...,...
535,4,5.842941,9.665402,0.282090
536,4,5.842941,9.665402,0.282090
537,4,5.842941,9.665402,0.282090
538,4,5.842941,9.665402,0.282090


In [257]:
cm_avg.loc[(cm_avg['origin_city'] == 'AtlantaGA') & (cm_avg['month'] == 4) | (cm_avg['month'] ==7)]

,origin_city,month,windspeeds,visibilities,precipitations,weather_Blowing_snow,weather_Clear,weather_Cloudy,weather_Fog,weather_Freezing_fog,...,weather_Patchy_light_rain,weather_Patchy_light_rain_with_thunder,weather_Patchy_light_snow,weather_Patchy_moderate_snow,weather_Patchy_rain_possible,weather_Patchy_sleet_possible,weather_Patchy_snow_possible,weather_Sunny,weather_Thundery_outbreaks_possible,weather_Torrential_rain_shower
3,AtlantaGA,4,5.842941,9.665402,0.282090,0.000066,0.185189,0.029779,0.001187,0.0,...,0.001979,0.001303,0.0,0.000066,0.039327,0.000099,0.000033,0.462355,0.003265,0.003891
6,AtlantaGA,7,6.706493,9.776554,0.222143,0.000000,0.189289,0.038751,0.000157,0.0,...,0.001243,0.000551,0.0,0.000000,0.029720,0.000000,0.000000,0.477997,0.002376,0.000629
18,CharlotteNC,7,8.881256,9.505846,0.565542,0.000000,0.156435,0.058110,0.000177,0.0,...,0.000354,0.007662,0.0,0.000000,0.034458,0.000000,0.000000,0.389140,0.012313,0.006998
30,ChicagoIL,7,8.407697,9.310065,0.616658,0.000000,0.143676,0.055178,0.003282,0.0,...,0.000499,0.005542,0.0,0.000000,0.036627,0.000000,0.000000,0.333088,0.006160,0.009965
42,DallasTX,7,8.853161,9.528696,0.516882,0.000000,0.174680,0.054505,0.000501,0.0,...,0.000324,0.011313,0.0,0.000000,0.024159,0.000000,0.000000,0.398179,0.014643,0.006629
54,DenverCO,7,7.564656,9.319330,0.499738,0.000000,0.123231,0.062902,0.002826,0.0,...,0.001539,0.001741,0.0,0.000000,0.056670,0.000000,0.000000,0.316958,0.007267,0.007973
66,HoustonTX,7,8.255295,9.217081,0.637633,0.000000,0.145095,0.061103,0.005564,0.0,...,0.000195,0.004685,0.0,0.000000,0.041386,0.000000,0.000000,0.307028,0.004295,0.011664
78,LosAngelesCA,7,7.524489,9.492053,0.375089,0.000000,0.158494,0.061243,0.001575,0.0,...,0.002166,0.002869,0.0,0.000000,0.039188,0.000000,0.000000,0.392494,0.003151,0.004360
90,PhoenixAZ,7,8.371819,9.623083,0.472911,0.000000,0.184878,0.061036,0.000548,0.0,...,0.006150,0.005307,0.0,0.000000,0.040059,0.000000,0.000000,0.387995,0.010783,0.003917
102,SanFranciscoCA,7,7.546285,9.285023,0.562572,0.000000,0.131613,0.067717,0.003602,0.0,...,0.002110,0.003129,0.0,0.000000,0.034423,0.000000,0.000000,0.320974,0.003020,0.008406


In [263]:
fnf.columns

Index(['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'visibilities', 'precipitations',
       'weather_Blowing_snow', 'weather_Clear', 'weather_Cloudy',
       'weather_Fog', 'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_sn

In [264]:
fnf.to_csv('../../../data/final_flights_cleaned_filled.csv')

In [265]:
fnt = fnf[['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'windspeeds', 'visibilities', 'precipitations',
        'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN',
       'weather_Blowing_snow', 'weather_Clear', 'weather_Cloudy',
       'weather_Fog', 'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_snow_showers', 'weather_Mist',
       'weather_Moderate_or_heavy_freezing_rain',
       'weather_Moderate_or_heavy_rain_shower',
       'weather_Moderate_or_heavy_rain_with_thunder',
       'weather_Moderate_or_heavy_sleet',
       'weather_Moderate_or_heavy_snow_showers',
       'weather_Moderate_or_heavy_snow_with_thunder', 'weather_Moderate_rain',
       'weather_Moderate_rain_at_times', 'weather_Moderate_snow',
       'weather_Overcast', 'weather_Partly_cloudy',
       'weather_Patchy_heavy_snow', 'weather_Patchy_light_drizzle',
       'weather_Patchy_light_rain', 'weather_Patchy_light_rain_with_thunder',
       'weather_Patchy_light_snow', 'weather_Patchy_moderate_snow',
       'weather_Patchy_rain_possible', 'weather_Patchy_sleet_possible',
       'weather_Patchy_snow_possible', 'weather_Sunny',
       'weather_Thundery_outbreaks_possible',
       'weather_Torrential_rain_shower', 'arr_delay']]

In [266]:
fnt

,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,...,weather_Patchy_light_rain_with_thunder,weather_Patchy_light_snow,weather_Patchy_moderate_snow,weather_Patchy_rain_possible,weather_Patchy_sleet_possible,weather_Patchy_snow_possible,weather_Sunny,weather_Thundery_outbreaks_possible,weather_Torrential_rain_shower,arr_delay
0,10397,14100,1,7,2018,534,125.0,666,9.934667,7.880984,...,0.000551,0.000000,0.000000,0.029720,0.000000,0.000000,0.477997,0.002376,0.000629,-9.0
1,10397,11057,1,7,2018,541,69.0,226,9.934667,7.880984,...,0.000551,0.000000,0.000000,0.029720,0.000000,0.000000,0.477997,0.002376,0.000629,6.0
2,10397,13303,1,7,2018,555,115.0,594,9.934667,7.880984,...,0.000551,0.000000,0.000000,0.029720,0.000000,0.000000,0.477997,0.002376,0.000629,-22.0
3,10397,13930,1,7,2018,623,136.0,606,9.934667,7.880984,...,0.000551,0.000000,0.000000,0.029720,0.000000,0.000000,0.477997,0.002376,0.000629,-26.0
4,10397,11298,1,7,2018,630,139.0,731,9.934667,7.880984,...,0.000551,0.000000,0.000000,0.029720,0.000000,0.000000,0.477997,0.002376,0.000629,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19853,14747,14107,30,4,2019,1445,168.0,1107,3.667013,7.880984,...,0.000534,0.000452,0.004023,0.031446,0.003407,0.000082,0.341393,0.003325,0.005747,-7.0
19854,14747,13930,30,4,2019,1515,249.0,1721,3.667013,7.880984,...,0.000534,0.000452,0.004023,0.031446,0.003407,0.000082,0.341393,0.003325,0.005747,-6.0
19855,14747,11298,30,4,2019,1533,239.0,1660,3.667013,7.880984,...,0.000534,0.000452,0.004023,0.031446,0.003407,0.000082,0.341393,0.003325,0.005747,70.0
19856,14747,14107,30,4,2019,1805,176.0,1107,3.667013,7.880984,...,0.000534,0.000452,0.004023,0.031446,0.003407,0.000082,0.341393,0.003325,0.005747,-5.0


In [267]:
fnt.to_csv('../../../data/theTrainingSet.csv')